In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [23]:
# rutas de archivos
kaggle_path = '/kaggle/input/petfinder-adoption-prediction/'
local_path = '../data/'
path = local_path
local_out_path = '../submissions/'
out_path = local_out_path

In [2]:
!ls /kaggle/input/petfinder-adoption-prediction/train

train.csv


In [10]:
# lectura de datasets
# Lee el CSV
df_train = pd.read_csv(path + "train/train.csv")
df_test = pd.read_csv(path + "test/test.csv")
sample_sub = pd.read_csv(path + "test/sample_submission.csv")


In [11]:
sample_sub.head()

,PetID,AdoptionSpeed
0,e2dfc2935,0
1,f153b465f,0
2,3c90f3f54,0
3,e02abc8a3,0
4,09f0df7d1,0


In [12]:
# Comparando datos de train y test
df_train.columns
df_test.columns


Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt'],
      dtype='object')

In [13]:
# Comparar la estructura de train y test
def compare_columns (df_train, df_test):
    print('Columnas en train', df_train.shape[1], " Columnas en test", df_test.shape[1])
    if df_train.shape[1] == df_test.shape[1]: 
        print("Misma cantidad de columnas")
        if df_train.columns.equals(df_test.columns):
            print('Mismo nombre de columnas')
            dtype_diff = (df_train.dtypes != (df_test.dtypes))
            if dtype_diff.any():
                print("Tipos de datos diferentes:")
                print(df_train.dtypes[dtype_diff].to_frame("train_dtype").join(
                      df_test.dtypes[dtype_diff].to_frame("test_dtype")))
            else:
                print('Mismos tipos de datos. Fin de la funcion')
                

        else:
            print("Columnas solo en train:", df_train.columns - df_test.columns)
            print("Columnas solo en test:", df_test.columns - df_train.columns)
    else:
        print('Cantidad de columnas diferentes')
        print('Columnas en train', df_train.shape[1], " Columnas en test", df_test.shape[1])

In [14]:
compare_columns(df_train.drop(columns=['AdoptionSpeed']), df_test)

Columnas en train 23  Columnas en test 23
Misma cantidad de columnas
Mismo nombre de columnas
Mismos tipos de datos. Fin de la funcion


# Exploración de las columnas

In [15]:
print(df_train.dtypes)
print(df_train.head())

Type               int64
Name              object
Age                int64
Breed1             int64
Breed2             int64
Gender             int64
Color1             int64
Color2             int64
Color3             int64
MaturitySize       int64
FurLength          int64
Vaccinated         int64
Dewormed           int64
Sterilized         int64
Health             int64
Quantity           int64
Fee                int64
State              int64
RescuerID         object
VideoAmt           int64
Description       object
PetID             object
PhotoAmt         float64
AdoptionSpeed      int64
dtype: object
   Type         Name  Age  Breed1  Breed2  Gender  Color1  Color2  Color3  \
0     2       Nibble    3     299       0       1       1       7       0   
1     2  No Name Yet    1     265       0       1       1       2       0   
2     1       Brisco    1     307       0       1       2       7       0   
3     1         Miko    4     307       0       2       1       2       0   
4

In [16]:
# Usar solo los numericos inicialmente
df_train_num = df_train.select_dtypes(include='number').copy()
df_train_num = df_train_num.drop(columns=['AdoptionSpeed'])
df_test_num = df_test.select_dtypes(include='number').copy()

In [50]:
df_train_num.head()

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt
0,2,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,41326,0,1.0
1,2,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,41401,0,2.0
2,1,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,41326,0,7.0
3,1,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150,41401,0,8.0
4,1,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,41326,0,3.0


# Modelo LightGBM

In [17]:
from lightgbm import LGBMClassifier

In [18]:
X_train = df_train_num
X_test = df_test_num  

#variable objetivo
label="AdoptionSpeed"
y_train = df_train[label]

# Entrenar el modelo
model=LGBMClassifier()
model.fit(X_train, y_train)

# Predecir sobre el mismo conjunto (solo para probar)
y_pred = model.predict(X_train)

# Evaluar precisión (accuracy)
accuracy = (y_pred == y_train).mean()
print("Precisión en train:", accuracy)



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001200 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 466
[LightGBM] [Info] Number of data points in the train set: 14993, number of used features: 19
[LightGBM] [Info] Start training from score -3.599182
[LightGBM] [Info] Start training from score -1.579412
[LightGBM] [Info] Start training from score -1.312082
[LightGBM] [Info] Start training from score -1.526163
[LightGBM] [Info] Start training from score -1.273213
Precisión en train: 0.5991462682585207


In [19]:

y_pred = model.predict(X_test)

In [24]:
# Crear archivo de submission
submission = pd.DataFrame({
    "PetID": df_test["PetID"],
    "AdoptionSpeed": y_pred
})
submission.to_csv(out_path + "submission.csv", index=False)
